In [18]:
library(rtweet)        # Used for extracting the tweets
library(tm)            # Text mining cleaning
library(stringr)       # Removing characters
library(qdapRegex)     # Removing URLs 
library(wordcloud2)    # Creating the wordcloud
library(httpuv)


Attaching package: 'qdapRegex'

The following object is masked from 'package:dplyr':

    explain



In [20]:
## store api keys (these are empty example values; replace with your own keys)
api_key <- ""
api_secret_key <- ""

## authenticate via web browser
token <- create_token(
  app = "",
  consumer_key = api_key,
  consumer_secret = api_secret_key)

Waiting for authentication in browser...
Press Esc/Ctrl + C to abort
Please point your browser to the following url: 
https://api.twitter.com/oauth/authenticate?oauth_token=4jZICgAAAAAA-dLgAAABcYUxoEU&permission=read
Authentication complete.


In [27]:
## most recent to least tweet order. Can set to 18000 I believe is the limit.
tweets_pab <- get_timelines(c(""), n = 8000)

In [28]:
## simple text cleaning
text <- str_c(tweets_pab$text, collapse = "")

In [29]:
## more text cleaning 
text <- 
  text %>%
  str_remove("\\n") %>%                   # remove linebreaks
  rm_twitter_url() %>%                    # Remove URLS
  rm_url() %>%
  str_remove_all("#\\S+") %>%             # Remove any hashtags
  str_remove_all("@\\S+") %>%             # Remove any @ mentions
  removeWords(stopwords("english")) %>%   # Remove common words (a, the, it etc.)
  removeNumbers() %>%
  stripWhitespace() %>%
  removeWords(c("amp"))                   # Final cleanup of other small changes

In [30]:
# Convert the data into a summary table
textCorpus <- 
  Corpus(VectorSource(text)) %>%
  TermDocumentMatrix() %>%
  as.matrix()

textCorpus <- sort(rowSums(textCorpus), decreasing=TRUE)
textCorpus <- data.frame(word = names(textCorpus), freq=textCorpus, row.names = NULL)

In [31]:
# build wordcloud 
wordcloud <- wordcloud2(data = textCorpus, minRotation = 0, maxRotation = 0, ellipticity = 0.6)
wordcloud

HTML widgets cannot be represented in plain text (need html)

In [32]:
## can group a bunch of twitter accounts together 

TweetsToWordcloud <- function(){
  
  tweets <- get_timelines(username, n = 3200)
  
  # Clean the data
  text <- str_c(tweets$text, collapse = "") %>%
  str_remove("\\n") %>%                   # remove linebreaks
  rm_twitter_url() %>%                    # Remove URLS
  rm_url() %>%
  str_remove_all("#\\S+") %>%             # Remove any hashtags
  str_remove_all("@\\S+") %>%             # Remove any @ mentions
  removeWords(stopwords("english")) %>%   # Remove common words (a, the, it etc.)
  removeNumbers() %>%
  stripWhitespace() %>%
  removeWords(c("amp"))                   # Final cleanup of other small changes
  
    # Convert the data into a summary table
  textCorpus <- 
    Corpus(VectorSource(text)) %>%
    TermDocumentMatrix() %>%
    as.matrix()
  
  textCorpus <- sort(rowSums(textCorpus), decreasing=TRUE)
  textCorpus <- data.frame(word = names(textCorpus), freq=textCorpus, row.names = NULL)
  
  wordcloud <- wordcloud2(data = textCorpus, minRotation = 0, maxRotation = 0, ellipticity = 0.6)
  return(wordcloud)
}